In [12]:
import os
from dotenv import load_dotenv
import json
from clickhouse_connect.driver.exceptions import ClickHouseError
import pandas as pd
import clickhouse_connect

load_dotenv('/home/jovyan/.env')

# Чтение параметров подключения
CLICKHOUSE_HOST = os.getenv('CLICKHOUSE_HOST')
CLICKHOUSE_PORT = int(os.getenv('CLICKHOUSE_PORT'))
CLICKHOUSE_DB = os.getenv('CLICKHOUSE_DB')
CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER')
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD')

try:
    client = clickhouse_connect.get_client(
        host=CLICKHOUSE_HOST,
        port=CLICKHOUSE_PORT,
        database=CLICKHOUSE_DB,
        username=CLICKHOUSE_USER,
        password=CLICKHOUSE_PASSWORD
    )
    print("Подключение к ClickHouse установлено!")
except Exception as e:
    print(f"Ошибка подключения: {str(e)}")

Подключение к ClickHouse установлено!


In [14]:
try:
    logins_count = client.query('SELECT COUNT(*) FROM logins').result_rows[0][0]
    users_count = client.query('SELECT COUNT(*) FROM user_info').result_rows[0][0]
    print(f"В таблице logins: {logins_count} записей")
    print(f"В таблице user_info: {users_count} записей")

    if logins_count > 0 or users_count > 0:
        print("Внимание: Данные уже есть в базе. Продолжайте с осторожностью.")

except Exception as e:
    print(f"Не удалось получить информацию из таблиц: {e}")

В таблице logins: 0 записей
В таблице user_info: 3 записей
Внимание: Данные уже есть в базе. Продолжайте с осторожностью.


In [ ]:
# 📥 Загрузка logins.csv
logins_csv_path = "/home/jovyan/data/logins.csv"
if not os.path.isfile(logins_csv_path):
    print(f"Файл не найден: {logins_csv_path}")
else:
    try:
        logins_df = pd.read_csv(logins_csv_path)
        logins_data = logins_df.to_dict('records')

        if logins_data:
            client.insert(
                'logins',
                logins_data,
            )
            print(f"Загружено {len(logins_data)} записей в таблицу logins.")
        else:
            print("В файле нет данных для загрузки.")
    except Exception as e:
        print(f"Ошибка при загрузке logins.csv: {e}")

In [ ]:
# Загрузка user_info.json
user_info_path = "/home/jovyan/data/user_info.json"
if not os.path.isfile(user_info_path):
    print(f"Файл не найден: {user_info_path}")
else:
        with open(user_info_path, 'r', encoding='utf-8') as f:
            user_info_data = json.load(f)
        df = pd.DataFrame(user_info_data)
        if not df.empty:
             try:
                client.insert(
                    'user_info',
                    df
                )
                print(f"✅ Загружено {len(user_info_data)} записей в таблицу user_info.")
             except  Exception as e:
                print(f"Ошибка при загрузке user_info.json: {e}")
        else:
            print("В файле нет данных для загрузки.")